__Tutorial for CUDA programming using Python__
==================================================

## Reduction

In computer science, the __reduction__ operator is a type of operator that is commonly used in parallel programming to reduce the elements of an array into a single result. 

+ vector norm
+ dot product

In [ ]:
# if you are using colab
# !pip install pycuda # pycuda installation

In [ ]:
import os
import time
import numpy as np

# --- PyCUDA initialization
import pycuda
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [ ]:
def nextpow2(x):  
    return 1 if x == 0 else 2**(x - 1).bit_length()

def getThreadsAndBlocks(kerId, num, maxBlocks, maxThreads):
    if kerId < 3:
        threads = nextpow2(num) if num < maxThreads else maxThreads
        blocks = int((num + threads - 1) / threads)
    else:
        threads = nextpow2(int((num+1)/2)) if num < maxThreads else maxThreads 
        blocks = int((num + 2*2*threads -1) / (2*threads))
    
    if kerId == 6:
        blocks = blocks if maxBlocks > blocks else maxBlocks

    return threads, blocks

In [ ]:
src_prKer1 = """

__global__ void product_reduction1(double* p, double*q, double*c, int NUM)
{
    extern __shared__ double sdata[];

    // load shared mem
    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x*blockDim.x + threadIdx.x;

    sdata[tid] = (i < NUM) ? p[i]*q[i] : 0;

    __syncthreads();

    // do reduction in shared mem
    for (unsigned int s=1; s < blockDim.x; s *= 2)
    {
        // modulo arithmetic is slow!
        if ((tid % (2*s)) == 0)
        {
            sdata[tid] += sdata[tid + s];
        }

        __syncthreads();
    }

    // write result for this block to global mem
    if (tid == 0) c[blockIdx.x] = sdata[0];
}
"""

In [ ]:
src_prKer6 = """

__global__ void product_reduction6(double* p, double*q, double*c, int NUM)
{
    unsigned int threadsPerBlocks = 512;
    bool numIsPow2 = true;
    extern __shared__ double sdata[];

    unsigned int tid = threadIdx.x;
    unsigned int idx = blockIdx.x*threadsPerBlocks*2 + threadIdx.x;
    unsigned int gridSize = threadsPerBlocks*2*gridDim.x;

    double temp = (double) 0;
    while(idx<NUM)
    {
        temp += (p[idx]*q[idx]);
        if(numIsPow2 || idx + threadsPerBlocks < NUM)
            temp += (p[idx+threadsPerBlocks]*q[idx+threadsPerBlocks]);
        idx += gridSize;
    }

    sdata[tid] = temp;
    __syncthreads();

    if(threadsPerBlocks>=512){if (tid<256) {sdata[tid] = temp = temp + sdata[tid+256];} __syncthreads();}	
    if(threadsPerBlocks>=256){if (tid<128) {sdata[tid] = temp = temp + sdata[tid+128];} __syncthreads();}	
    if(threadsPerBlocks>=128){if (tid< 64) {sdata[tid] = temp = temp + sdata[tid+ 64];} __syncthreads();}

    if(tid<32)
    {
        volatile double* smem = sdata;
        if(threadsPerBlocks >= 64){ smem[tid] = temp = temp + smem[tid+32];}
        if(threadsPerBlocks >= 32){ smem[tid] = temp = temp + smem[tid+16];}
        if(threadsPerBlocks >= 16){ smem[tid] = temp = temp + smem[tid+ 8];}
        if(threadsPerBlocks >=  8){ smem[tid] = temp = temp + smem[tid+ 4];}
        if(threadsPerBlocks >=  4){ smem[tid] = temp = temp + smem[tid+ 2];}
        if(threadsPerBlocks >=  2){ smem[tid] = temp = temp + smem[tid+ 1];}
    }

    if(tid==0)
        c[blockIdx.x] = sdata[0];
}

"""

In [ ]:
# e_start = pycuda.driver.Event()
# e_stop = pycuda.driver.Event()

N = 2**27

# thread, blocks, shared memory size
threads, blocks = getThreadsAndBlocks(1, N, 64, 512)

smems = 2*threads*8 if threads <= 32 else threads*8
print(threads, blocks, smems)

# block and grid dimensions
blockDim  = (threads, 1, 1)
gridDim   = (blocks, 1, 1)

In [ ]:
h_a = np.random.uniform(-1, 1, size=N).astype(np.float64)
h_b = np.random.uniform(-1, 1, size=N).astype(np.float64)
h_cache = np.zeros(blocks).astype(np.float64) 

# copy to device
d_a = pycuda.gpuarray.to_gpu(h_a)
d_b = pycuda.gpuarray.to_gpu(h_b)
d_cache = pycuda.gpuarray.zeros(blocks, dtype='float64')

In [ ]:
# cuda compile ...

print('kernel build')
module_ker1 = pycuda.compiler.SourceModule(source=src_prKer1)
# 
dev_dot_ker1 = module_ker1.get_function("product_reduction1")

##
module_ker6 = pycuda.compiler.SourceModule(source=src_prKer6)
# 
dev_dot_ker6 = module_ker6.get_function("product_reduction6")



In [ ]:
# e_start.record()
print('reduction : gpu kernel 1')

dev_dot_ker1(d_a, d_b, d_cache, np.int32(N), block=blockDim, grid=gridDim, shared=smems)

t_start = time.time()

dev_dot_ker1(d_a, d_b, d_cache, np.int32(N), block=blockDim, grid=gridDim, shared=smems)
h_cache = d_cache.get()
c_ = np.sum(h_cache)

elapsed = time.time() - t_start

pycuda.driver.Context.synchronize()

print(c_)
print("Processing time = {:f}".format(elapsed))

print("#"*64)

print('reduction : gpu kernel 6')

# temp
dev_dot_ker6(d_a, d_b, d_cache, np.int32(N), block=blockDim, grid=gridDim, shared=smems)

t_start = time.time()

dev_dot_ker6(d_a, d_b, d_cache, np.int32(N), block=blockDim, grid=gridDim, shared=smems)
h_cache = d_cache.get()
c_ = np.sum(h_cache)

elapsed = time.time() - t_start

pycuda.driver.Context.synchronize()

print(c_)
print("Processing time = {:f}".format(elapsed))


print("#"*64)

# print('reduction : gpuarray dot')
# t_start = time.time()
# d_c = pycuda.gpuarray.vdot(d_a, d_b)
# elapsed = time.time() - t_start
# c_ = d_c.get()
# pycuda.driver.Context.synchronize()

# print(c_)

# print("Processing time = {:f}".format(elapsed))

# print("#"*64)

print('reduction : cpu')
t_start = time.time()
c_= np.dot(h_a, h_b)
elapsed = time.time() - t_start

print(c_)
#secs = e_start.time_till(e_stop) * 1e-3
print("Processing time = {:f}".format(elapsed))

In [ ]:
print("#"*64)

print('reduction : pycuda gpuarray dot')

d_c = pycuda.gpuarray.dot(d_a, d_b)

t_start = time.time()
d_c = pycuda.gpuarray.dot(d_a, d_b)
c_ = d_c.get()
elapsed = time.time() - t_start

pycuda.driver.Context.synchronize()
print(c_)

# secs = e_start.time_till(e_stop) * 1e-3
print("Processing time = {:f}".format(elapsed))
